In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive # creating a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
%cd /mydrive/FYP/Ensemble

In [ ]:
!pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt  # install dependencies
!git clone https://github.com/ultralytics/yolov5.git  # clone repo
%cd yolov5
!python -c "from yolov5.utils.google_utils import gdrive_download; gdrive_download('1d1A_fKtXIKW1zCf_Lyqj3Q-zrISLlVbZ', 'yolov5s.pt')"  # download yolov5s.pt
%cd ..
!wget https://github.com/facebookresearch/detectron2/archive/refs/tags/v0.6.1.zip
!unzip v0.6.1.zip
%cd detectron2-0.6.1/
!pip install -e .

In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd
import os

class CustomDataset(Dataset):
    def __init__(self, data_dir, transforms):
        self.transforms = transforms
        
        # Load the image filepaths and annotations from a CSV file
        self.data = pd.read_csv(os.path.join(data_dir, 'annotations.csv'))
        self.img_dir = os.path.join(data_dir, 'images')
        
    def __getitem__(self, idx):
        # Load the image and its annotations
        img_path = os.path.join(self.img_dir, self.data.iloc[idx, 0])
        img = Image.open(img_path).convert('RGB')
        boxes = torch.tensor(eval(self.data.iloc[idx, 1]), dtype=torch.float32)
        labels = torch.tensor(eval(self.data.iloc[idx, 2]), dtype=torch.int64)
        
        # Apply transforms
        img, boxes, labels = self.transforms(img, boxes, labels)
        
        # Return a dictionary containing the image and its annotations
        return {'image': img, 'boxes': boxes, 'labels': labels}
    
    def __len__(self):
        return len(self.data)

In [ ]:
import torch

def calculate_accuracy(model, data_loader, device):
    model.eval()
    total_correct = 0
    total_images = 0
    
    with torch.no_grad():
        for batch in data_loader:
            images, targets = batch['image'].to(device), batch['targets']
            outputs = model(images)
            
            # Iterate over each image in the batch
            for i in range(len(targets)):
                # Get the ground truth and predicted boxes and labels for the current image
                gt_boxes = targets[i]['boxes'].to(device)
                gt_labels = targets[i]['labels'].to(device)
                pred_boxes = outputs[i]['boxes']
                pred_labels = outputs[i]['labels']
                
                # Calculate the number of correctly detected objects
                correct = 0
                for j in range(len(gt_labels)):
                    if gt_labels[j] in pred_labels and \
                       (pred_boxes[pred_labels == gt_labels[j]] == gt_boxes[j]).any():
                        correct += 1
                total_correct += correct
                total_images += 1
    
    accuracy = total_correct / total_images
    return accuracy


In [ ]:
def imShow(path):
    import cv2
    import matplotlib.pyplot as plt
    %matplotlib inline
    image = cv2.imread(path)
    height, width = image.shape[:2]
    resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)
    fig = plt.gcf()
    fig.set_size_inches(18, 10)
    plt.axis('off')
    plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
import torch
from torch.utils.data import DataLoader
from yolov5.models import YOLOv5

# Define the YOLOv5 model
model_yolov5 = YOLOv5(num_classes=num_classes)

# Define the optimizer and the loss function
optimizer = torch.optim.Adam(model_yolov5.parameters(), lr=lr)
loss_fn = YOLOv5Loss(num_classes=num_classes, strides=model_yolov5.strides)

# Define the data loader
dataset = CustomDataset(annotations_file)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Train the YOLOv5 model
for epoch in range(num_epochs):
    for images, targets in data_loader:
        images = images.to(device)
        targets = [{k: v.to(device) for k, v in target.items()} for target in targets]

        optimizer.zero_grad()

        outputs = model_yolov5(images)

        loss = sum([loss_fn(outputs[i], targets[i]) for i in range(len(outputs))])
        loss.backward()

        optimizer.step()
yolov5_accuracy = calculate_accuracy(model_yolov5, data_loader, device)
print(f'YOLOv5 Accuracy: {yolov5_accuracy}')
imShow('chart.png')

In [ ]:
import cv2
import numpy as np
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, launch
from detectron2 import model_zoo
from detectron2.data import build_detection_train_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultPredictor

# Define the dataset directory and the annotations file
dataset_dir = 'path/to/dataset'
annotations_file = 'path/to/annotations.json'

# Register the custom dataset
register_coco_instances('custom_dataset_train', {}, annotations_file, dataset_dir)

# Define the configuration
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_50_C4_3x.yaml'))
cfg.DATASETS.TRAIN = ('custom_dataset_train',)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.DEVICE = 'cuda'
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-Detection/faster_rcnn_R_50_C4_3x.yaml')
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.02
cfg.SOLVER.MAX_ITER = 1000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes

# Train the Faster R-CNN model
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

imShow('chart.png')
# faster_rcnn_accuracy = calculate_accuracy(model_yolov5, cfg.DATALOADER, cfg.MODEL.DEVICE)
# print(f'Faster RCNN Accuracy: {faster_rcnn_accuracy}')


In [ ]:
from PIL import Image
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Load the test image
image = Image.open('path/to/test/image.jpg')

# Use the Faster R-CNN model to detect objects
cfg.MODEL.WEIGHTS = 'path/to/faster_rcnn/model.pth'
predictor = DefaultPredictor(cfg)
output_frcnn = predictor(image)

# Use the YOLOv5 model to detect objects
model_yolov5.to(device)
model_yolov5.eval()
with torch.no_grad():
    image_tensor = transforms(image).unsqueeze(0).to(device)
    output_yolov5 = model_yolov5(image_tensor)
    output_yolov5 = non_max_suppression(output_yolov5, conf_thres=0.5, iou_thres=0.5)

# Visualize the results from Faster R-CNN
metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
v = Visualizer(np.array(image), metadata=metadata, scale=1.0)
v = v.draw_instance_predictions(output_frcnn['instances'].to('cpu'))
Image.fromarray(v.get_image())

# Visualize the results from YOLOv5
image_yolov5 = plot_one_box(output_yolov5[0][0][:4].cpu().numpy(), np.array(image), color=[0,0,255], line_thickness=2)
Image.fromarray(image_yolov5)


In [ ]:
results = [output_frcnn, output_yolov5]
result_ensemble = ensemble_results(results)


In [ ]:
# import matplotlib.pyplot as plt

# def plot_results(image, result):
#     fig, ax = plt.subplots(figsize=(10, 10))
#     ax.imshow(image)

#     for box in result.xyxy[0]:
#         x1, y1, x2, y2 = box[:4].tolist()
#         score = box[4].item()
#         width = max(2, int(image.width / 400))
#         ax.add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor='red', lw=width


In [ ]:
import matplotlib.pyplot as plt

def plot_results(image, instances):
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(image)
    colors = plt.cm.hsv(np.linspace(0, 1, len(instances)+1)).tolist()

    # Plot each instance with a unique color
    for i, inst in enumerate(instances):
        color = colors[i]
        bbox = inst['bbox']
        label = inst['label']
        score = inst['score']
        ax.add_patch(plt.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3] - bbox[1],
                                    fill=False, edgecolor=color, linewidth=2))
        ax.text(bbox[0], bbox[1] - 2, f'{label} {score:.2f}', fontsize=12,
                color=color, ha='center', va='top')
    plt.axis('off')
    plt.show()


In [ ]:
# Visualize the ensemble results
plot_results(image, result_ensemble)

In [ ]:
# ---------------------old----------------------------------------

In [ ]:
# import torch

# model_yolov5 = torch.hub.load('ultralytics/yolov5', 'custom', path_or_model='yolov5s.pt')

In [ ]:
# import cv2
# import numpy as np
# from detectron2.config import get_cfg
# from detectron2.engine import DefaultPredictor

# # Create the configuration for the Faster R-CNN model
# cfg = get_cfg()
# cfg.merge_from_file("detectron2-0.6.1/configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model

# # Download the weights for the Faster R-CNN model
# cfg.MODEL.WEIGHTS = "detectron2-0.6.1/model_final_faster_rcnn.pth"

# # Create the predictor for the Faster R-CNN model
# model_frcnn = DefaultPredictor(cfg)

In [ ]:
# from PIL import Image

# image_path = 'path/to/image.jpg'
# image = Image.open(image_path)

# result_yolov5 = model_yolov5(image)
# result_frcnn = model_frcnn(cv2.imread(image_path)[:, :, ::-1])

In [ ]:
# def ensemble_results(results):
#     # Combine the bounding box coordinates and confidence scores from each model
#     boxes = []
#     scores = []
#     for result in results:
#         boxes.append(result.xyxy[0])
#         scores.append(result.xyxy[0][:, 4])
#     boxes = torch.stack(boxes)
#     scores = torch.stack(scores)

#     # Compute the weighted average of the bounding box coordinates and confidence scores
#     weights = scores / torch.sum(scores, dim=0, keepdim=True)
#     weighted_boxes = torch.sum(weights.unsqueeze(-1) * boxes, dim=0)
#     weighted_scores = torch.mean(scores, dim=0)

#     # Create a new result object with the ensemble results
#     result = results[0].clone()
#     result.xyxy[0] = weighted_boxes
#     result.pandas().xyxy[0]['confidence'] = weighted_scores.cpu().numpy()

In [ ]:
# results = [result_yolov5, result_frcnn]
# result_ensemble = ensemble_results(results)

In [ ]:
# import matplotlib.pyplot as plt

# def plot_results(image, result):
#     fig, ax = plt.subplots(figsize=(10, 10))
#     ax.imshow(image)

#     for box in result.xyxy[0]:
#         x1, y1, x2, y2 = box[:4].tolist()
#         score = box[4].item()
#         width = max(2, int(image.width / 400))
#         ax.add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor='red', lw=width))
#         text = f'{score:.2f}'
#         ax.text(x1, y1, text, fontsize=max(14, width * 2), color='white', ha='left', va='top',
#                 bbox=dict(facecolor='red', alpha=0.8, lw=0))
#     plt.axis('off')
#     plt.show()

In [ ]:
# plot_results(image, result_ensemble)